# Incremental margin algorithm for large margin classifiers

In [20]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc
from sklearn import datasets
from sklearn.svm import LinearSVC
from sklearn import preprocessing
from sklearn.datasets import load_digits
import time
from sklearn import svm
from sklearn.model_selection import GridSearchCV 
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from scipy.stats import sem
from numpy import linalg as LA
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

## Calculating the margin

In [21]:
def compute_margin(X, y, w, b):
    margin = []
    for i in range(y.shape[0]):
        margin.append((y[i]*(np.dot(X[i,:], w)+b))/sqrt(sum(w**2))) 
    if min(margin) >= 0:
        return min(margin)
    return 0

## L0 norm

In [22]:
def L0_norm(w, threshold):
    l0_norm = 0
    for wi in w:
        if abs(wi) > threshold:
            l0_norm += 1
    return l0_norm

# ELM

In [23]:
def sigmoid(z):
    sigmoid = 1.0/(1.0 + np.exp(-z))
    return sigmoid 

## ELM with IM P inf

In [24]:
import random
from sklearn.base import BaseEstimator, ClassifierMixin

class ELM_IMA(BaseEstimator, ClassifierMixin):

    # Inicialization of important parameters 
    def __init__(self, n_neurons, eta=0.1, lambda_param=0.01, delta_margin=10^-3,
                 IMA_iterations=10, max_updates=10000, threshold=0.1):
        self.n_neurons = n_neurons              # Neurons of hidden layer osf ELM
        self.eta = eta                          # Learning rate
        self.lambda_param = lambda_param        # Param important of soft margin
        self.delta_margin = delta_margin        # (1 + delta_margin) * fixed margin defines the minimum next margin of IMA
        self.IMA_iterations = IMA_iterations    # Maximum number of iterations of IMA
        self.max_updates = max_updates          # Maximum number of updates in one execution of FMP
        self.w = np.array([])                   # Vector of weights of the last layer of the ELM obtained after the training of the IMA
        self.w_elm = np.array([])               # Vector of weights of the last layer of the ELM obtained after the normal training of ELM
        self.H = np.array([])                   # H matrix of ELM (obtained with training data)
        self.Z = np.array([])                   # Z matrix of ELM
        self.b = 0
        self.threshold = threshold

    # Fixed Margin Algorithm    
    def FMP_algorithm(self, X, y, w_init, b_init, fixed_margin, idx, s):
        t = 0
        iterations = 0
        w = w_init
        b = b_init
        w_norm_1 = LA.norm(w, ord=1)
        last_t = -1
        lambda_t = 0
        alpha = np.zeros((X.shape[0]))
        while True:
            last_t = t
            e=0
            for k in range(0, y.shape[0]):
                i = int(idx[k])
                if(y[i]*(np.dot(X[i,:], w)+b) <= fixed_margin * w_norm_1 - self.lambda_param * alpha[i]):
                    if w_norm_1 != 0:
                        lambda_t = 1 - (self.eta*fixed_margin)/w_norm_1
                    else:
                        lambda_t = 1
                    alpha = alpha * lambda_t
                    alpha[i] = alpha[i] + self.eta    
                    w = w - self.eta * (fixed_margin * np.sign(w) - y[i] * X[i,:])
                    w_norm_1 = LA.norm(w, ord=1)
                    b = b + self.eta*y[i]
                    t += 1
                    e += 1
                    if k > s:
                        s += 1
                        j = s
                    else:
                        j=e
                    idx[k], idx[j] = idx[j], idx[k]
            iterations += 1
            if (t > self.max_updates or last_t == t):
                break
        if t<= self.max_updates:
            convergence=1
        else:
            convergence=0
        return w, b, convergence, t, iterations, idx, s

    # IMA Algorithm
    def IM_algorithm(self, X, y):
        self.w = np.zeros(self.H.shape[1])
        self.ws = [] 
        self.bs = [] 
        self.ws.append(self.w)
        self.bs.append(self.b)
        fixed_margin = 0#compute_margin(X, y, self.w_elm, self.b)
        t = 0
        convergence = 1
        updates=0
        iterations=0
        margin=[]
        margin.append(fixed_margin)
        idx = np.linspace(0, y.shape[0]-1, y.shape[0])
        s=0
        l=0
        while convergence==1 and t<self.IMA_iterations:
            w, b, convergence, updates_, iterations_, idx, s = self.FMP_algorithm(X, y, self.w, self.b, fixed_margin, idx, s)
            if convergence == 1:
                self.w = w
                self.b = b
                self.ws.append(self.w)
                self.bs.append(self.b)
            updates += updates_
            iterations += iterations_
            norm_w = LA.norm(w, ord=1)
            gamma1 = []
            gamma2 = []
            for i in range(0, y.shape[0]):
                if y[i] == 1:
                    gamma1.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
                else:
                    gamma2.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
            gamma1 = np.array(gamma1)
            gamma2 = np.array(gamma2)
            gamma1 = gamma1[gamma1>=0]
            gamma2 = gamma2[gamma2>=0]
            if len(gamma1) == 0:
                min_gamma1 = 0
            else:
                min_gamma1 = min(gamma1)
            if len(gamma2) == 0:
                min_gamma2 = 0
            else:
                min_gamma2 = min(gamma2)
            fixed_margin = max([(min_gamma1 + min_gamma2)/2, (1+self.delta_margin)*fixed_margin])
            #margin.append(compute_margin(X, y, self.w, self.b))
            t += 1
            l0_norm = 0
            remove = []
            for i in range(len(self.w)):
                if abs(self.w[i]) < self.threshold * self.w.max():
                    remove.append(i)
            self.w = np.delete(self.w, remove)
            self.Z = np.delete(self.Z, remove, axis=1)
            self.H = np.delete(self.H, remove, axis=1)
            X = np.delete(X, remove, axis=1)
        return t, updates, iterations

    # Function that manage the training of IMA ELM
    def fit(self, X, y, Z=[]):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        X = X_new
        n = X.shape[1]
        if len(Z) == 0:
            self.Z = np.array([random.uniform(-0.5, 0.5) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        else:
            self.Z = Z
        self.H = np.tanh(np.dot(X, self.Z))
        #w = np.dot(np.linalg.pinv(self.H), y)  
        #self.w_elm = w.reshape((w.shape[0],))
        iterations_IMA, updates, iterations = self.IM_algorithm(self.H, y) 
        return iterations_IMA, updates, iterations
            
    # Function to apply IMA ELM model
    def predict(self, X, use_IMA_w=True):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H = np.tanh(np.dot(X_new, self.Z))
        if use_IMA_w == True:
            y_predicted = np.sign(np.dot(H, self.w) + self.b)
        #else:
        #    y_predicted = np.sign(np.dot(H,  self.w_elm))
        y_predicted[y_predicted==0]=-1
        return y_predicted

## Commom ELM

In [25]:
class ELM(BaseEstimator, ClassifierMixin):
         
    def __init__(self, n_neurons):
        self.n_neurons = n_neurons
    
    def fit(self, X, y, Z=[]):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        X = X_new
        n = X.shape[1]
        if len(Z) == 0:
            self.Z = np.array([random.uniform(-0.5, 0.5) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        else:
            self.Z = Z
        self.H = np.tanh(np.dot(X, self.Z))
        self.w = np.dot(np.linalg.pinv(self.H), y)  
        return self.w, self.H, self.Z
            
    def predict(self, X):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H = np.tanh(np.dot(X_new, self.Z))
        y_predicted = np.sign(np.dot(H, self.w))
        y_predicted[y_predicted==0]=1
        return y_predicted

## Function to Capture Results

In [26]:
def grid_l(X, y, p):
    # GridSearch for lambda and learning rate of IMA ELM
    parameters = {'lambda_param':np.linspace(0.01, 10, 50)}
    clf = ELM_IMA(n_neurons=p, delta_margin=10^-3, IMA_iterations=10, max_updates=10000)
    clf = GridSearchCV(clf, parameters, scoring='accuracy', cv=5, verbose=0)
    clf.fit(X, y)
    return clf.best_params_['lambda_param']

In [27]:
def results(X, y, n_splits, p, eta, IMA_iterations, lambda_param):      
    print(f'Parameters: p={p}, eta={eta}, lambda={lambda_param}')
    # Stratified k fold cross validation
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=72)
    i=0
    
    train_accuracy_IM_ELM_02 = np.zeros(n_splits)
    test_accuracy_IM_ELM_02 = np.zeros(n_splits)
    margin_IM_ELM_02 = np.zeros(n_splits)
    updates_02 = np.zeros(n_splits)
    iterations_FMP_02 = np.zeros(n_splits) 
    iterations_IMA_02 = np.zeros(n_splits)
    hidden_neurons_02 = np.zeros(n_splits)
    
    train_accuracy_IM_ELM_01 = np.zeros(n_splits)
    test_accuracy_IM_ELM_01 = np.zeros(n_splits)
    margin_IM_ELM_01 = np.zeros(n_splits)
    updates_01 = np.zeros(n_splits)
    iterations_FMP_01 = np.zeros(n_splits) 
    iterations_IMA_01 = np.zeros(n_splits)
    hidden_neurons_01 = np.zeros(n_splits)
    
    train_accuracy_IM_ELM_001 = np.zeros(n_splits)
    test_accuracy_IM_ELM_001 = np.zeros(n_splits)
    margin_IM_ELM_001 = np.zeros(n_splits)
    updates_001 = np.zeros(n_splits)
    iterations_FMP_001 = np.zeros(n_splits) 
    iterations_IMA_001 = np.zeros(n_splits)
    hidden_neurons_001 = np.zeros(n_splits)
    
    train_accuracy_IM_ELM_0001 = np.zeros(n_splits)
    test_accuracy_IM_ELM_0001 = np.zeros(n_splits)
    margin_IM_ELM_0001 = np.zeros(n_splits)
    updates_0001 = np.zeros(n_splits)
    iterations_FMP_0001 = np.zeros(n_splits) 
    iterations_IMA_0001 = np.zeros(n_splits)
    hidden_neurons_0001 = np.zeros(n_splits)
    
    train_accuracy_ELM = np.zeros(n_splits)
    test_accuracy_ELM = np.zeros(n_splits)
    margin_ELM = np.zeros(n_splits)
        
    for train_index, test_index in kf.split(X, y):
        X_train = X[train_index,:]
        X_test = X[test_index,:]
        y_train = y[train_index]
        y_test = y[test_index]
               
        # ELM
        clf = ELM(n_neurons=p)
        clf.fit(X_train, y_train)
        y_hat=clf.predict(X_test)
        y_hat_train=clf.predict(X_train)
        margin_ELM[i] = compute_margin(clf.H[:,:], y_train, clf.w, 0)
        train_accuracy_ELM[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_ELM[i] = accuracy_score(y_test, y_hat)
        Z = clf.Z
        
        # ELM-IMA 0.2
        clf_02 = ELM_IMA(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3,
                      IMA_iterations=IMA_iterations, max_updates=10000, threshold=0.2)
        iterations_IMA_02[i], updates_02[i], iterations_FMP_02[i] = clf_02.fit(X_train, y_train, Z)
        y_hat=clf_02.predict(X_test, use_IMA_w = True)
        y_hat_train=clf_02.predict(X_train, use_IMA_w = True)
        margin_IM_ELM_02[i] = compute_margin(clf_02.H[:,:], y_train, clf_02.w, clf_02.b)
        train_accuracy_IM_ELM_02[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM_02[i] = accuracy_score(y_test, y_hat)
        hidden_neurons_02[i] = len(clf_02.w)
        
        # ELM-IMA 0.1
        clf_01 = ELM_IMA(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3,
                      IMA_iterations=IMA_iterations, max_updates=10000, threshold=0.1)
        iterations_IMA_01[i], updates_01[i], iterations_FMP_01[i] = clf_01.fit(X_train, y_train, Z)
        y_hat=clf_01.predict(X_test, use_IMA_w = True)
        y_hat_train=clf_01.predict(X_train, use_IMA_w = True)
        margin_IM_ELM_01[i] = compute_margin(clf_01.H[:,:], y_train, clf_01.w, clf_01.b)
        train_accuracy_IM_ELM_01[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM_01[i] = accuracy_score(y_test, y_hat)
        hidden_neurons_01[i] = len(clf_01.w)
        
                
        # ELM-IMA 0.01
        clf_001 = ELM_IMA(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3,
                      IMA_iterations=IMA_iterations, max_updates=10000, threshold=0.01)
        iterations_IMA_001[i], updates_001[i], iterations_FMP_001[i] = clf_001.fit(X_train, y_train, Z)
        y_hat=clf_001.predict(X_test, use_IMA_w = True)
        y_hat_train=clf_001.predict(X_train, use_IMA_w = True)
        margin_IM_ELM_001[i] = compute_margin(clf_001.H[:,:], y_train, clf_001.w, clf_001.b)
        train_accuracy_IM_ELM_001[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM_001[i] = accuracy_score(y_test, y_hat)
        hidden_neurons_001[i] = len(clf_001.w)

        # ELM-IMA 0.001
        clf_0001 = ELM_IMA(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3,
                      IMA_iterations=IMA_iterations, max_updates=10000, threshold=0.001)
        iterations_IMA_0001[i], updates_0001[i], iterations_FMP_0001[i] = clf_0001.fit(X_train, y_train, Z)
        y_hat=clf_0001.predict(X_test, use_IMA_w = True)
        y_hat_train=clf_0001.predict(X_train, use_IMA_w = True)
        margin_IM_ELM_0001[i] = compute_margin(clf_0001.H[:,:], y_train, clf_0001.w, clf_0001.b)
        train_accuracy_IM_ELM_0001[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM_0001[i] = accuracy_score(y_test, y_hat)
        hidden_neurons_0001[i] = len(clf_0001.w)       
        i+=1
    
    print("********* Results ELM-IMA 0.2**************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM_02.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM_02.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM_02.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM_02.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP_02.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP_02.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA_02.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA_02.std()))
    print("Updates: " + '{:.4f}'.format(updates_02.mean())+ "+/-" + '{:.4f}'.format(updates_02.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM_02.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM_02.std()))
    print("Hidden Neurons: " + '{:.2f}'.format(hidden_neurons_02.mean()) + "+/-" + '{:.2f}'.format(hidden_neurons_02.std()))
    
    print("********* Results ELM-IMA 0.1**************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM_01.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM_01.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM_01.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM_01.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP_01.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP_01.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA_01.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA_01.std()))
    print("Updates: " + '{:.4f}'.format(updates_01.mean())+ "+/-" + '{:.4f}'.format(updates_01.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM_01.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM_01.std()))
    print("Hidden Neurons: " + '{:.2f}'.format(hidden_neurons_01.mean()) + "+/-" + '{:.2f}'.format(hidden_neurons_01.std()))
    
    print("********* Results ELM-IMA 0.01**************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM_001.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM_001.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM_001.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM_001.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP_001.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP_001.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA_001.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA_001.std()))
    print("Updates: " + '{:.4f}'.format(updates_001.mean())+ "+/-" + '{:.4f}'.format(updates_001.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM_001.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM_001.std()))
    print("Hidden Neurons: " + '{:.2f}'.format(hidden_neurons_001.mean()) + "+/-" + '{:.2f}'.format(hidden_neurons_001.std()))
    
    print("********* Results ELM-IMA 0.001**************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM_0001.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM_0001.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM_0001.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM_0001.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP_0001.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP_0001.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA_0001.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA_0001.std()))
    print("Updates: " + '{:.4f}'.format(updates_0001.mean())+ "+/-" + '{:.4f}'.format(updates_0001.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM_0001.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM_0001.std()))
    print("Hidden Neurons: " + '{:.2f}'.format(hidden_neurons_0001.mean()) + "+/-" + '{:.2f}'.format(hidden_neurons_0001.std()))
            
    print("********* Results ELM **************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_ELM.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_ELM.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_ELM.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_ELM.std()))
    print("Margin: " + '{:.9f}'.format(margin_ELM.mean())+  "+/-" + '{:.9f}'.format(margin_ELM.std()))  

In [28]:
def run(X,y):
    # Normalizing data:
    X = preprocessing.normalize(X, axis=0)
    n = len(X)
    if n>1000:
        n=1000
    for p in [n]:
        l = grid_l(X, y, p=p)
        print(f"Experimento com {p} neurônios:" )
        results(X, y, n_splits=10, p=p, eta=0.1, IMA_iterations=20, lambda_param=l)
        print("\n\n")

## Application on Iris Dataset


In [29]:
iris = datasets.load_iris()
X = iris.data
# setosa - 0, versicolor - 1, virginica - 2  
y = iris.target 
# O problema agora possui apenas as classes y=-1 e y=1
y[y>0] = 1
y[y==0] = -1
run(X, y)

Experimento com 150 neurônios:
Parameters: p=150, eta=0.1, lambda=0.01
********* Results ELM-IMA 0.2**************
Acc train: 0.9985+/-0.0030
Acc test: 0.9933+/-0.0200
Iterations: 695.0000+/-120.1815
Iterations IMA: 20.0000+/-0.0000
Updates: 3882.4000+/-461.5589
Margin: 0.039109676+/-0.019798457
Hidden Neurons: 21.10+/-3.53
********* Results ELM-IMA 0.1**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 666.7000+/-219.8240
Iterations IMA: 20.0000+/-0.0000
Updates: 3550.4000+/-1064.6719
Margin: 0.050484005+/-0.008452262
Hidden Neurons: 31.90+/-9.74
********* Results ELM-IMA 0.01**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 667.2000+/-190.3932
Iterations IMA: 20.0000+/-0.0000
Updates: 4052.7000+/-1333.2539
Margin: 0.055758337+/-0.003721812
Hidden Neurons: 48.80+/-15.24
********* Results ELM-IMA 0.001**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 754.2000+/-369.7650
Iterations IMA: 20.0000+/-0.0

## Application on Synthetic Dataset

In [30]:
synthetic_dataset = pd.read_csv('data/synthetic_dataset/synthetic_control.data', sep="\s+",  header=None, engine='python')
X = synthetic_dataset.to_numpy()
y = np.concatenate((np.ones(100), np.ones(200)*-1, np.ones(100), np.ones(100)*-1,np.ones(100)))
run(X, y)

Experimento com 600 neurônios:
Parameters: p=600, eta=0.1, lambda=0.01
********* Results ELM-IMA 0.2**************
Acc train: 0.9611+/-0.1130
Acc test: 0.9600+/-0.1091
Iterations: 905.3000+/-388.8429
Iterations IMA: 20.0000+/-0.0000
Updates: 7764.9000+/-4060.8808
Margin: 0.019935174+/-0.012761510
Hidden Neurons: 121.60+/-45.71
********* Results ELM-IMA 0.1**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9967+/-0.0067
Iterations: 488.7000+/-394.5337
Iterations IMA: 20.0000+/-0.0000
Updates: 4301.9000+/-3285.1707
Margin: 0.028672535+/-0.009452073
Hidden Neurons: 279.60+/-71.78
********* Results ELM-IMA 0.01**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9967+/-0.0067
Iterations: 315.6000+/-112.0323
Iterations IMA: 20.0000+/-0.0000
Updates: 3474.8000+/-1601.1547
Margin: 0.030319826+/-0.004875722
Hidden Neurons: 462.20+/-40.98
********* Results ELM-IMA 0.001**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9983+/-0.0050
Iterations: 483.3000+/-302.9647
Iterations IMA: 20.0000

## Application on Robot  Dataset



In [31]:
robot_dataset = pd.read_csv('data/robot/lp4_data.csv', delimiter =',')
X = robot_dataset.to_numpy().reshape([117,90])
y = np.concatenate((np.ones(24), np.ones(117-24)*-1))
run(X, y)

Experimento com 117 neurônios:
Parameters: p=117, eta=0.1, lambda=3.2720408163265304
********* Results ELM-IMA 0.2**************
Acc train: 0.8139+/-0.0438
Acc test: 0.7538+/-0.0745
Iterations: 229.8000+/-94.5059
Iterations IMA: 12.2000+/-6.6903
Updates: 6625.7000+/-4676.9033
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 12.40+/-8.69
********* Results ELM-IMA 0.1**************
Acc train: 0.8186+/-0.0459
Acc test: 0.7697+/-0.0517
Iterations: 374.9000+/-225.5635
Iterations IMA: 12.7000+/-6.3411
Updates: 6881.9000+/-4826.2452
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 25.60+/-13.32
********* Results ELM-IMA 0.01**************
Acc train: 0.8224+/-0.0558
Acc test: 0.7697+/-0.0517
Iterations: 785.3000+/-511.6542
Iterations IMA: 16.5000+/-4.7170
Updates: 9252.0000+/-6887.5619
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 60.50+/-15.03
********* Results ELM-IMA 0.001**************
Acc train: 0.7815+/-0.0913
Acc test: 0.7121+/-0.1558
Iterations: 560.5000+/-221.5275
Iterations IMA

## Application on Mushroom Dataset

In [32]:
df = pd.read_csv('data/Mushroom/agaricus-lepiota.data', delimiter =',', header=None)
df = df.replace("?", np.nan) 
df = df.dropna() 
y = df[0].to_numpy()
X = df.drop([0], axis='columns')
X = pd.get_dummies(X).to_numpy()
y[np.where(y=='e')] = -1
y[np.where(y=='p')] = 1
y = np.array(y.tolist())
run(X, y)

Experimento com 1000 neurônios:
Parameters: p=1000, eta=0.1, lambda=2.4565306122448978
********* Results ELM-IMA 0.2**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 79.0000+/-25.0000
Iterations IMA: 20.0000+/-0.0000
Updates: 1672.8000+/-452.0473
Margin: 0.070108589+/-0.021139063
Hidden Neurons: 363.00+/-55.26
********* Results ELM-IMA 0.1**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9998+/-0.0005
Iterations: 72.0000+/-18.9631
Iterations IMA: 20.0000+/-0.0000
Updates: 1874.0000+/-761.5978
Margin: 0.069700504+/-0.021119205
Hidden Neurons: 561.80+/-110.90
********* Results ELM-IMA 0.01**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9998+/-0.0005
Iterations: 64.0000+/-11.8406
Iterations IMA: 20.0000+/-0.0000
Updates: 1750.1000+/-668.3682
Margin: 0.060044241+/-0.023035318
Hidden Neurons: 844.60+/-82.65
********* Results ELM-IMA 0.001**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9998+/-0.0005
Iterations: 64.4000+/-12.2000
Iterations IMA: 2

## Application on Ionosphere Dataset

In [33]:
ionosphere_dataset = pd.read_csv('data/Ionosphere/ionosphere.data', names=list(range(0,35)), sep=',')
y = ionosphere_dataset[34].to_numpy()
X = ionosphere_dataset.drop([34], axis='columns').to_numpy()
y[np.where(y=='g')] = 1
y[np.where(y=='b')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 351 neurônios:
Parameters: p=351, eta=0.1, lambda=2.2526530612244895
********* Results ELM-IMA 0.2**************
Acc train: 0.8968+/-0.0293
Acc test: 0.8490+/-0.0528
Iterations: 212.5000+/-38.7692
Iterations IMA: 20.0000+/-0.0000
Updates: 3654.3000+/-390.3207
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 99.40+/-10.04
********* Results ELM-IMA 0.1**************
Acc train: 0.9095+/-0.0185
Acc test: 0.8747+/-0.0601
Iterations: 240.0000+/-31.7112
Iterations IMA: 20.0000+/-0.0000
Updates: 4771.3000+/-624.9499
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 162.00+/-10.51
********* Results ELM-IMA 0.01**************
Acc train: 0.9133+/-0.0182
Acc test: 0.8631+/-0.0676
Iterations: 278.2000+/-18.5138
Iterations IMA: 20.0000+/-0.0000
Updates: 6485.4000+/-664.5048
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 260.30+/-9.84
********* Results ELM-IMA 0.001**************
Acc train: 0.9006+/-0.0201
Acc test: 0.8348+/-0.0596
Iterations: 295.1000+/-27.4680
Iterations IMA: 20

## Application on Banknote Dataset

In [34]:
# read in banknote authentication set
banknotes = pd.read_csv('data/banknote/data_banknote_authentication.txt', names=['variance', 'skewness', 'curtosis', 'entropy', 'class'])
# convert to array
X = banknotes[['variance', 'skewness', 'curtosis', 'entropy']].to_numpy()
y = banknotes[['class']].to_numpy()
y[np.where(y==0)] = -1
run(X, y)

Experimento com 1000 neurônios:
Parameters: p=1000, eta=0.1, lambda=0.41775510204081634
********* Results ELM-IMA 0.2**************
Acc train: 0.9831+/-0.0116
Acc test: 0.9839+/-0.0121
Iterations: 903.5000+/-211.4092
Iterations IMA: 20.0000+/-0.0000
Updates: 20890.5000+/-4639.8608
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 415.50+/-62.36
********* Results ELM-IMA 0.1**************
Acc train: 0.9840+/-0.0063
Acc test: 0.9840+/-0.0102
Iterations: 918.1000+/-498.3572
Iterations IMA: 20.0000+/-0.0000
Updates: 20863.3000+/-10981.9353
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 609.40+/-54.28
********* Results ELM-IMA 0.01**************
Acc train: 0.9837+/-0.0078
Acc test: 0.9840+/-0.0166
Iterations: 1039.5000+/-404.4805
Iterations IMA: 20.0000+/-0.0000
Updates: 24598.5000+/-9159.3877
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 805.30+/-52.67
********* Results ELM-IMA 0.001**************
Acc train: 0.9826+/-0.0065
Acc test: 0.9898+/-0.0088
Iterations: 1116.2000+/-196.9705


## Application on Wine Dataset

In [35]:
wine_dataset = pd.read_csv('data/wine/wine.data', names=['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315', 'Proline'])
# convert to array
y = wine_dataset[['Class']].to_numpy()
X = wine_dataset.drop("Class",axis='columns').to_numpy()
y[np.where(y==3)] = 1
y[np.where(y==2)] = -1
run(X, y)

Experimento com 178 neurônios:
Parameters: p=178, eta=0.1, lambda=0.01
********* Results ELM-IMA 0.2**************
Acc train: 0.9944+/-0.0169
Acc test: 0.9605+/-0.0369
Iterations: 1367.9000+/-626.4989
Iterations IMA: 20.0000+/-0.0000
Updates: 11066.5000+/-4635.1118
Margin: 0.005234620+/-0.003339995
Hidden Neurons: 67.30+/-14.14
********* Results ELM-IMA 0.1**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9663+/-0.0371
Iterations: 1285.5000+/-629.0283
Iterations IMA: 20.0000+/-0.0000
Updates: 11594.1000+/-5374.1011
Margin: 0.006817214+/-0.003032389
Hidden Neurons: 98.30+/-21.47
********* Results ELM-IMA 0.01**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9775+/-0.0372
Iterations: 1103.8000+/-762.0687
Iterations IMA: 20.0000+/-0.0000
Updates: 10590.6000+/-5601.6617
Margin: 0.006139575+/-0.002803331
Hidden Neurons: 145.40+/-19.23
********* Results ELM-IMA 0.001**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9552+/-0.0545
Iterations: 1027.4000+/-356.7647
Iterations IMA: 20

## Application on WDBC Dataset

In [36]:
wdbc_dataset = pd.read_csv('data/WDBC/wdbc.data', names=list(range(0,32)))
# convert to array
y = wdbc_dataset[1].to_numpy()
X = wdbc_dataset.drop([0, 1],axis='columns').to_numpy()
y[np.where(y=='B')] = 1
y[np.where(y=='M')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 569 neurônios:
Parameters: p=569, eta=0.1, lambda=1.6410204081632653
********* Results ELM-IMA 0.2**************
Acc train: 0.9668+/-0.0154
Acc test: 0.9648+/-0.0208
Iterations: 254.0000+/-36.1967
Iterations IMA: 20.0000+/-0.0000
Updates: 3354.3000+/-510.4071
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 144.20+/-29.76
********* Results ELM-IMA 0.1**************
Acc train: 0.9697+/-0.0126
Acc test: 0.9684+/-0.0172
Iterations: 277.6000+/-68.1985
Iterations IMA: 20.0000+/-0.0000
Updates: 3953.2000+/-1026.9285
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 230.20+/-36.41
********* Results ELM-IMA 0.01**************
Acc train: 0.9691+/-0.0107
Acc test: 0.9614+/-0.0189
Iterations: 370.3000+/-102.4325
Iterations IMA: 20.0000+/-0.0000
Updates: 6041.6000+/-1489.5728
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 318.90+/-59.03
********* Results ELM-IMA 0.001**************
Acc train: 0.9705+/-0.0102
Acc test: 0.9649+/-0.0175
Iterations: 337.0000+/-88.1601
Iterations IM

## Application on Sonar Dataset

In [37]:
sonar_dataset = pd.read_csv('data/sonar/sonar.all-data', names=list(range(0,61)), sep=',')
y = sonar_dataset[60].to_numpy()
X = sonar_dataset.drop([60], axis='columns').to_numpy()
y[np.where(y=='R')] = 1
y[np.where(y=='M')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 208 neurônios:
Parameters: p=208, eta=0.1, lambda=9.796122448979592
********* Results ELM-IMA 0.2**************
Acc train: 0.8595+/-0.0252
Acc test: 0.7783+/-0.0836
Iterations: 143.0000+/-14.8862
Iterations IMA: 20.0000+/-0.0000
Updates: 1539.4000+/-133.3921
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 56.50+/-12.49
********* Results ELM-IMA 0.1**************
Acc train: 0.8819+/-0.0276
Acc test: 0.7490+/-0.0840
Iterations: 125.3000+/-18.6657
Iterations IMA: 20.0000+/-0.0000
Updates: 1526.4000+/-196.1658
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 90.80+/-10.21
********* Results ELM-IMA 0.01**************
Acc train: 0.8579+/-0.0285
Acc test: 0.7729+/-0.1068
Iterations: 143.7000+/-25.4639
Iterations IMA: 20.0000+/-0.0000
Updates: 1797.5000+/-193.7014
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 154.80+/-7.22
********* Results ELM-IMA 0.001**************
Acc train: 0.8621+/-0.0389
Acc test: 0.7638+/-0.0867
Iterations: 146.0000+/-20.9857
Iterations IMA: 20.0

## Application on Pima Dataset

In [38]:
pima_dataset = pd.read_csv('data/diabetes/diabetes.csv', sep=",", engine='python')
y = pima_dataset['Outcome'].to_numpy()
X = pima_dataset.drop(['Outcome'], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 768 neurônios:
Parameters: p=768, eta=0.1, lambda=9.388367346938775
********* Results ELM-IMA 0.2**************
Acc train: 0.7323+/-0.0274
Acc test: 0.7085+/-0.0536
Iterations: 516.1000+/-346.1894
Iterations IMA: 20.0000+/-0.0000
Updates: 9889.0000+/-1762.6670
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 191.90+/-61.06
********* Results ELM-IMA 0.1**************
Acc train: 0.7406+/-0.0509
Acc test: 0.7527+/-0.0645
Iterations: 492.2000+/-449.4765
Iterations IMA: 20.0000+/-0.0000
Updates: 10665.1000+/-2725.0792
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 304.30+/-99.25
********* Results ELM-IMA 0.01**************
Acc train: 0.7622+/-0.0142
Acc test: 0.7526+/-0.0365
Iterations: 481.6000+/-417.5936
Iterations IMA: 20.0000+/-0.0000
Updates: 12164.4000+/-4277.6175
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 529.50+/-100.65
********* Results ELM-IMA 0.001**************
Acc train: 0.7613+/-0.0267
Acc test: 0.7670+/-0.0453
Iterations: 380.7000+/-245.6046
Iterati

## Application on Statlog Dataset

In [39]:
statlog_dataset = pd.read_csv('data/statlog/heart.dat', sep=" ", header=None, engine='python')
y = statlog_dataset[13].to_numpy()
X = statlog_dataset.drop([13], axis='columns').to_numpy()
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 270 neurônios:
Parameters: p=270, eta=0.1, lambda=9.184489795918367
********* Results ELM-IMA 0.2**************
Acc train: 0.8202+/-0.0536
Acc test: 0.8074+/-0.1071
Iterations: 108.2000+/-24.5471
Iterations IMA: 20.0000+/-0.0000
Updates: 1875.3000+/-201.3490
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 66.80+/-11.69
********* Results ELM-IMA 0.1**************
Acc train: 0.8317+/-0.0203
Acc test: 0.7963+/-0.0727
Iterations: 115.7000+/-38.5488
Iterations IMA: 20.0000+/-0.0000
Updates: 2047.8000+/-310.7037
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 111.40+/-17.47
********* Results ELM-IMA 0.01**************
Acc train: 0.8346+/-0.0153
Acc test: 0.8037+/-0.0723
Iterations: 111.0000+/-20.7654
Iterations IMA: 20.0000+/-0.0000
Updates: 2262.5000+/-334.1590
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 184.50+/-11.83
********* Results ELM-IMA 0.001**************
Acc train: 0.8391+/-0.0226
Acc test: 0.8185+/-0.0914
Iterations: 111.1000+/-27.8763
Iterations IMA: 20

## Application on Mammographic Dataset

In [40]:
mammo = pd.read_csv('data/mammographic/mammographic_masses.data', sep=",", header=None, engine='python')
mammo = mammo.replace("?", np.nan)
mammo = mammo.dropna()
y = mammo[5].to_numpy()
X = mammo.drop([5], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 830 neurônios:
Parameters: p=830, eta=0.1, lambda=4.495306122448979
********* Results ELM-IMA 0.2**************
Acc train: 0.7114+/-0.0954
Acc test: 0.7169+/-0.0926
Iterations: 294.0000+/-108.9413
Iterations IMA: 20.0000+/-0.0000
Updates: 15554.0000+/-4081.2096
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 199.00+/-22.13
********* Results ELM-IMA 0.1**************
Acc train: 0.7438+/-0.0561
Acc test: 0.7578+/-0.0712
Iterations: 354.2000+/-154.9966
Iterations IMA: 20.0000+/-0.0000
Updates: 19994.0000+/-4495.7206
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 327.00+/-29.07
********* Results ELM-IMA 0.01**************
Acc train: 0.7763+/-0.0418
Acc test: 0.7723+/-0.0648
Iterations: 206.9000+/-86.8406
Iterations IMA: 20.0000+/-0.0000
Updates: 17874.3000+/-5344.4924
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 623.60+/-24.61
********* Results ELM-IMA 0.001**************
Acc train: 0.7475+/-0.0723
Acc test: 0.7470+/-0.0803
Iterations: 213.8000+/-44.5259
Iteration

## Application on Haberman Dataset

In [41]:
haberman = pd.read_csv('data/haberman/haberman.data', sep=",", header=None, engine='python')
y = haberman[3].to_numpy()
X = haberman.drop([3], axis='columns').to_numpy()
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 306 neurônios:
Parameters: p=306, eta=0.1, lambda=0.21387755102040817
********* Results ELM-IMA 0.2**************
Acc train: 0.7190+/-0.0639
Acc test: 0.7220+/-0.0506
Iterations: 169.3000+/-63.2788
Iterations IMA: 3.8000+/-1.6613
Updates: 17088.9000+/-3829.2253
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 51.40+/-55.45
********* Results ELM-IMA 0.1**************
Acc train: 0.7357+/-0.0013
Acc test: 0.7353+/-0.0094
Iterations: 209.8000+/-106.7612
Iterations IMA: 3.5000+/-0.9220
Updates: 18265.8000+/-4967.2179
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 102.50+/-69.17
********* Results ELM-IMA 0.01**************
Acc train: 0.7360+/-0.0017
Acc test: 0.7353+/-0.0094
Iterations: 185.5000+/-57.7274
Iterations IMA: 3.3000+/-1.2689
Updates: 19133.6000+/-4173.9754
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 241.00+/-45.16
********* Results ELM-IMA 0.001**************
Acc train: 0.7357+/-0.0021
Acc test: 0.7353+/-0.0094
Iterations: 177.9000+/-53.7707
Iterations I

## Application on Transfusion Dataset

In [42]:
transfusion = pd.read_csv('data/transfusion/transfusion.data', sep=",", engine='python')
y = transfusion["whether he/she donated blood in March 2007"].to_numpy()
X = transfusion.drop(["whether he/she donated blood in March 2007"], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 748 neurônios:
Parameters: p=748, eta=0.1, lambda=0.21387755102040817
********* Results ELM-IMA 0.2**************
Acc train: 0.7620+/-0.0005
Acc test: 0.7621+/-0.0041
Iterations: 106.9000+/-85.4207
Iterations IMA: 1.0000+/-0.0000
Updates: 10105.8000+/-70.6227
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 748.00+/-0.00
********* Results ELM-IMA 0.1**************
Acc train: 0.7620+/-0.0005
Acc test: 0.7621+/-0.0041
Iterations: 106.9000+/-85.4207
Iterations IMA: 1.0000+/-0.0000
Updates: 10105.8000+/-70.6227
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 748.00+/-0.00
********* Results ELM-IMA 0.01**************
Acc train: 0.7620+/-0.0005
Acc test: 0.7621+/-0.0041
Iterations: 106.9000+/-85.4207
Iterations IMA: 1.0000+/-0.0000
Updates: 10105.8000+/-70.6227
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 748.00+/-0.00
********* Results ELM-IMA 0.001**************
Acc train: 0.7620+/-0.0005
Acc test: 0.7621+/-0.0041
Iterations: 106.9000+/-85.4207
Iterations IMA: 1.000

## Application on Australian Credit 

In [43]:
australian = pd.read_csv('data/australian_credit/australian.dat', header=None, sep=" ", engine='python')
australian = australian.replace("?", np.nan)
australian = australian.dropna()
y = australian[14].to_numpy()
X = australian.drop([14], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 690 neurônios:
Parameters: p=690, eta=0.1, lambda=9.388367346938775
********* Results ELM-IMA 0.2**************
Acc train: 0.8385+/-0.0305
Acc test: 0.8348+/-0.0551
Iterations: 244.9000+/-125.8876
Iterations IMA: 20.0000+/-0.0000
Updates: 5993.5000+/-1433.5135
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 190.00+/-32.88
********* Results ELM-IMA 0.1**************
Acc train: 0.8269+/-0.0379
Acc test: 0.8391+/-0.0397
Iterations: 188.0000+/-95.3216
Iterations IMA: 20.0000+/-0.0000
Updates: 6432.0000+/-1892.4567
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 324.90+/-38.05
********* Results ELM-IMA 0.01**************
Acc train: 0.8403+/-0.0289
Acc test: 0.8275+/-0.0516
Iterations: 174.6000+/-58.6467
Iterations IMA: 20.0000+/-0.0000
Updates: 7194.0000+/-1687.9887
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 502.60+/-36.78
********* Results ELM-IMA 0.001**************
Acc train: 0.8329+/-0.0260
Acc test: 0.8319+/-0.0527
Iterations: 173.0000+/-48.8938
Iterations IM

## Application on Breast Cancer Dataset

In [44]:
breast = pd.read_csv('data/breast/breast.data', header=None, sep=",", engine='python')
breast = breast.replace("?", np.nan)
breast = breast.dropna()
y = breast[10].to_numpy()
X = breast.drop([0, 10], axis='columns').to_numpy()
y[np.where(y==4)] = 1
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 683 neurônios:
Parameters: p=683, eta=0.1, lambda=6.534081632653061
********* Results ELM-IMA 0.2**************
Acc train: 0.9608+/-0.0098
Acc test: 0.9619+/-0.0229
Iterations: 133.3000+/-33.0607
Iterations IMA: 20.0000+/-0.0000
Updates: 1446.7000+/-223.0220
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 190.50+/-37.12
********* Results ELM-IMA 0.1**************
Acc train: 0.9597+/-0.0114
Acc test: 0.9531+/-0.0254
Iterations: 168.9000+/-61.3897
Iterations IMA: 20.0000+/-0.0000
Updates: 1887.1000+/-424.8944
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 289.30+/-59.35
********* Results ELM-IMA 0.01**************
Acc train: 0.9623+/-0.0092
Acc test: 0.9531+/-0.0261
Iterations: 140.1000+/-39.4574
Iterations IMA: 20.0000+/-0.0000
Updates: 1863.6000+/-405.3690
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 467.70+/-39.37
********* Results ELM-IMA 0.001**************
Acc train: 0.9632+/-0.0091
Acc test: 0.9546+/-0.0266
Iterations: 162.0000+/-53.3648
Iterations IMA: 2

## Application on Spam Dataset

In [45]:
#spam = pd.read_csv('data/spam/spambase.data', header=None, sep=",", engine='python')
#y = spam[57].to_numpy()
#X = spam.drop([57], axis='columns').to_numpy()
#y[np.where(y==0)] = -1
#y = np.array(y.tolist())
#run(X, y)

## Application on Glass Dataset

In [46]:
# Loading dataset:
headers = ["Id", "RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe", "Class"]
df = pd.read_csv("~/Documents/UFMG/Graduation/10/Reconhecimento de padrões/list/pattern-recognition-exercises/list_5/databases/glass.csv", names = headers)
X = df.drop("Class", axis=1)
X = X.drop("Id", axis=1)
y = df["Class"]
X = X.to_numpy()
y = y.to_numpy()
y[np.where(y>1)] = -1
run(X, y)

Experimento com 214 neurônios:
Parameters: p=214, eta=0.1, lambda=4.6991836734693875
********* Results ELM-IMA 0.2**************
Acc train: 0.6791+/-0.0184
Acc test: 0.6818+/-0.0311
Iterations: 356.5000+/-240.2541
Iterations IMA: 6.8000+/-5.0557
Updates: 10023.5000+/-2620.3880
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 28.70+/-15.61
********* Results ELM-IMA 0.1**************
Acc train: 0.6444+/-0.1073
Acc test: 0.6303+/-0.1107
Iterations: 247.4000+/-87.7932
Iterations IMA: 10.9000+/-7.5690
Updates: 7644.4000+/-4298.6631
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 62.40+/-26.90
********* Results ELM-IMA 0.01**************
Acc train: 0.6807+/-0.0231
Acc test: 0.6773+/-0.0180
Iterations: 298.6000+/-191.7035
Iterations IMA: 7.8000+/-5.2307
Updates: 10619.2000+/-2857.2152
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 149.30+/-42.16
********* Results ELM-IMA 0.001**************
Acc train: 0.6864+/-0.0274
Acc test: 0.6825+/-0.0442
Iterations: 252.9000+/-69.0151
Iterations IM